# 初步窥探Kangle 之 Titanic 生还预测入门篇

<b> 流程：
- 数据预处理  
-  EDA
- 交叉验证  
- 建模预测

## 读取数据，观察数据

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
train_set = pd.read_csv('titanic_train.csv')
test_set = pd.read_csv('test.csv')

In [3]:
display(train_set.head())
display(test_set.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


较之train_set.columns, test.columns少了一列 Survived.  
先用 $0$ 补上这列

In [4]:
test_set['Survived'] = 0

In [5]:
train_set.info()
print('-'*44)
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
--------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-

## 数据预处理 
可以看出 train_set 和 test_set 中有float64(2), int64(5), object(5)，且 'Age', 'Fare', 'Cabin' 均有 null 值。  
**字段含义**
* Age: Age

* Cabin: Cabin

* Embarked: Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

* Fare: Passenger Fare

* Name: Name

* Parch: Number of Parents/Children Aboard

* Pclass: Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)

* Sex: Sex

* Sibsp: Number of Siblings/Spouses Aboard

* Survived: Survival (0 = No; 1 = Yes)

* Ticket: Ticket Number

In [6]:
train_set.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
test_set.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
Survived         0
dtype: int64

In [8]:
train_set.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

由于 ‘Cabin’ 缺失值太多，选择舍去该特征.  
 由于 Embarked 特征为 object，采用数量多的值补充缺失值
 接下来将特征数字化

In [9]:
from sklearn.preprocessing import LabelEncoder
def encode_features(data_set, feature_names):
    '''将data_set中的feature_name转换为数字'''
    for feature_name in feature_names:
        le = LabelEncoder()
        le.fit(data_set[feature_name])
        encoded_column = le.transform(data_set[feature_name])
        data_set[feature_name] = encoded_column
    return data_set

### Encode features & handle missing data & create new feature

In [10]:
total_set = [train_set, test_set]
feature2encode = ['Sex', 'Embarked']
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for i in range(2):
    
    total_set[i].Fare = total_set[i].Fare.fillna(total_set[i].Fare.mean()) # 均值填充 Fare 缺失值
    total_set[i].Age = total_set[i].Age.fillna(total_set[i].Age.mean())  # 均值填充 Age 缺失值
    total_set[i].Embarked = total_set[i].Embarked.fillna('S')  # 出现最多 S 填充缺失值
    
    # creating new features
    total_set[i]['FamilySize'] = total_set[i]['Parch'] + total_set[i]['SibSp']
    
    total_set[i]['Title'] = total_set[i].Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # replace titles with a more common title or as Rare
    total_set[i]['Title'] = total_set[i]['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    total_set[i]['Title'] = total_set[i]['Title'].replace('Mlle', 'Miss')
    total_set[i]['Title'] = total_set[i]['Title'].replace('Ms', 'Miss')
    total_set[i]['Title'] = total_set[i]['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    total_set[i]['Title'] = total_set[i]['Title'].map(titles)
    # filling NaN with 0, to get safe
    total_set[i]['Title'] = total_set[i]['Title'].fillna(0)
#     丢弃无用的特征
    total_set[i] = total_set[i].drop(['Name','Ticket','Cabin', 'Parch', 'SibSp'], axis=1)
#     encode features
    total_set[i] = encode_features(total_set[i], feature2encode)
    
    display(total_set[i].head())
    total_set[i].info()
    print('-'*44)


,PassengerId,Survived,Pclass,Sex,Age,Fare,Embarked,FamilySize,Title
0,1,0,3,1,22.0,7.2500,2,1,1
1,2,1,1,0,38.0,71.2833,0,1,3
2,3,1,3,0,26.0,7.9250,2,0,2
3,4,1,1,0,35.0,53.1000,2,1,3
4,5,0,3,1,35.0,8.0500,2,0,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null int64
Age            891 non-null float64
Fare           891 non-null float64
Embarked       891 non-null int64
FamilySize     891 non-null int64
Title          891 non-null int64
dtypes: float64(2), int64(7)
memory usage: 62.7 KB
--------------------------------------------


,PassengerId,Pclass,Sex,Age,Fare,Embarked,Survived,FamilySize,Title
0,892,3,1,34.5,7.8292,1,0,0,1
1,893,3,0,47.0,7.0000,2,0,1,3
2,894,2,1,62.0,9.6875,1,0,0,1
3,895,3,1,27.0,8.6625,2,0,0,1
4,896,3,0,22.0,12.2875,2,0,2,3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Sex            418 non-null int64
Age            418 non-null float64
Fare           418 non-null float64
Embarked       418 non-null int64
Survived       418 non-null int64
FamilySize     418 non-null int64
Title          418 non-null int64
dtypes: float64(2), int64(7)
memory usage: 29.5 KB
--------------------------------------------


## 凭感觉鲁莽选择特征  
<b>确认训练集

In [11]:
x_train = total_set[0].drop(['PassengerId', 'Survived'], axis=1)
y_train = total_set[0].Survived
x_test = total_set[1].drop(['PassengerId','Survived'], axis=1)

## 使用随机森林建模预测

In [12]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=1, n_estimators=100, min_samples_split=4, min_samples_leaf=2)
rfc.fit(x_train, y_train)
y_pre = rfc.predict(x_test)

D:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [13]:
data_to_submit = pd.DataFrame({
    'PassengerId': test_set.PassengerId,
    'Survived': y_pre
})

In [14]:
data_to_submit

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [ ]:
# data_to_submit.to_csv("results.csv", index=False)

<b> 输出提交结果  
    
# Score: 0.78468